In [1]:
import sys
import numpy as np
import pandas as pd

In [2]:
path = sys.path[0]+'/'
labels = pd.read_csv(path+'labels.csv')
data = pd.read_csv(path+'train.csv')

In [12]:
labels['attribute_id'][[i for i in range(0,12)]]

0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
Name: attribute_id, dtype: int64

In [13]:
labels

,attribute_id,attribute_name
0,0,country::afghanistan
1,1,country::alamania
2,2,country::algeria
3,3,country::arabia
4,4,country::armenia
...,...,...
3469,3469,tags::writing
3470,3470,tags::writing implements
3471,3471,tags::zebras
3472,3472,tags::zeus


In [15]:
def label_indexer_fine(labels_dataframe):
    split_labels_dict = dict(attribute_id=[], attr_tier1=[], attr_tier2=[], attr_tier3=[])
    for i in range(labels_dataframe.shape[0]):
        tem = [item.strip() for split_list in [item_.split(';') 
                                               for item_ in labels_dataframe['attribute_name'][i].split('::')] 
               for item in split_list]
        split_labels_dict['attribute_id'].append(labels_dataframe['attribute_id'][i])
        split_labels_dict['attr_tier1'].append(tem[0])
        split_labels_dict['attr_tier2'].append(tem[1])
        try:
            split_labels_dict['attr_tier3'].append(tem[2])
        except:
            split_labels_dict['attr_tier3'].append('None')
    split_labels = pd.DataFrame(split_labels_dict, 
                                columns=['attribute_id', 'attr_tier1', 'attr_tier2', 'attr_tier3'])
    
    tier1 = dict()
    tier2 = dict()
    counting_dict = dict()
    attr2indexing = dict()
    indexing2attr = dict()
    label_indexing_list = []
    for i_1, item1 in enumerate(sorted(list(set(list(split_labels['attr_tier1']))))):
        tier1[item1] = i_1
        tier2[item1] = dict()
        list_tem = sorted(list(set(list(split_labels['attr_tier2'][split_labels['attr_tier1']==item1]))))
        for i_2, item2 in enumerate(list_tem):
            tier2[item1][item2] = i_2 + 1
        counting_dict[item1] = np.ones(len(list_tem), dtype='int')
    for idx in range(split_labels.shape[0]):
        tier1_idx = tier1[split_labels['attr_tier1'][idx]]
        tier2_idx = tier2[split_labels['attr_tier1'][idx]][split_labels['attr_tier2'][idx]]
        tier3_idx = counting_dict[split_labels['attr_tier1'][idx]][tier2_idx-1]
        counting_dict[split_labels['attr_tier1'][idx]][tier2_idx-1] += 1
        label_indexing_list.append([tier1_idx, tier2_idx, tier3_idx])
        attr2indexing[split_labels['attribute_id'][idx]] = [tier1_idx, tier2_idx, tier3_idx]
        indexing2attr[str([tier1_idx, tier2_idx, tier3_idx])] = split_labels['attribute_id'][idx]
    labels_indexing_df = split_labels.copy()
    labels_indexing_df['indexing'] = label_indexing_list
    return labels_indexing_df, attr2indexing, indexing2attr

In [10]:
def label_indexer_coarse(labels_dataframe):
    split_labels_dict = dict(attribute_id=[], attr_tier1=[], attr_tier2=[])
    for i in range(labels_dataframe.shape[0]):
        tem = [item.strip() for item in labels_dataframe['attribute_name'][i].split('::')] 
        split_labels_dict['attribute_id'].append(labels_dataframe['attribute_id'][i])
        split_labels_dict['attr_tier1'].append(tem[0])
        split_labels_dict['attr_tier2'].append(tem[1])
        
    split_labels = pd.DataFrame(split_labels_dict, 
                                columns=['attribute_id', 'attr_tier1', 'attr_tier2'])
    
    tier1 = dict()
    tier2 = dict()
    attr2indexing = dict()
    indexing2attr = dict()
    label_indexing_list = []
    for i_1, item1 in enumerate(sorted(list(set(list(split_labels['attr_tier1']))))):
        assert len(list(set(list(split_labels['attr_tier2'][split_labels['attr_tier1']==item1])))) \
        == len(list(split_labels['attr_tier2'][split_labels['attr_tier1']==item1]))
        tier1[item1] = i_1
        tier2[item1] = dict()
        list_tem = list(split_labels['attr_tier2'][split_labels['attr_tier1']==item1])
        for i_2, item2 in enumerate(list_tem):
            tier2[item1][item2] = i_2 + 1
    for idx in range(split_labels.shape[0]):
        tier1_idx = tier1[split_labels['attr_tier1'][idx]]
        tier2_idx = tier2[split_labels['attr_tier1'][idx]][split_labels['attr_tier2'][idx]]
        label_indexing_list.append([tier1_idx, tier2_idx])
        attr2indexing[split_labels['attribute_id'][idx]] = [tier1_idx, tier2_idx]
        indexing2attr[str([tier1_idx, tier2_idx])] = split_labels['attribute_id'][idx]
    labels_indexing_df = split_labels.copy()
    labels_indexing_df['indexing'] = label_indexing_list
    return labels_indexing_df, attr2indexing, indexing2attr

In [12]:
labels_indexing_df, attr2indexing, indexing2attr = label_indexer_coarse(labels)

In [13]:
labels_indexing_df

,attribute_id,attr_tier1,attr_tier2,indexing
0,0,country,afghanistan,"[0, 1]"
1,1,country,alamania,"[0, 2]"
2,2,country,algeria,"[0, 3]"
3,3,country,arabia,"[0, 4]"
4,4,country,armenia,"[0, 5]"
...,...,...,...,...
3469,3469,tags,writing,"[4, 764]"
3470,3470,tags,writing implements,"[4, 765]"
3471,3471,tags,zebras,"[4, 766]"
3472,3472,tags,zeus,"[4, 767]"


In [16]:
labels_indexing_df2, attr2indexing2, indexing2attr2 = label_indexer_fine(labels)

In [19]:
labels_indexing_df['indexing'][labels_indexing_df2['attr_tier1']=='medium']

786        [3, 1]
787        [3, 2]
788        [3, 3]
789        [3, 4]
790        [3, 5]
          ...    
2701    [3, 1916]
2702    [3, 1917]
2703    [3, 1918]
2704    [3, 1919]
2705    [3, 1920]
Name: indexing, Length: 1920, dtype: object

In [21]:
[len(labels_indexing_df['indexing'][labels_indexing_df['attr_tier1']==catagory]) 
 for catagory in sorted(list(set(list(labels_indexing_df['attr_tier1']))))]

[100, 681, 5, 1920, 768]

In [18]:
labels_indexing_df2['indexing'][labels_indexing_df2['attr_tier1']=='medium']

786        [3, 1, 1]
787        [3, 2, 1]
788        [3, 3, 1]
789        [3, 4, 1]
790        [3, 5, 1]
            ...     
2701    [3, 1803, 1]
2702    [3, 1804, 1]
2703    [3, 1805, 1]
2704    [3, 1806, 1]
2705    [3, 1807, 1]
Name: indexing, Length: 1920, dtype: object

In [4]:
def split_attr_names(attr_name_str):
    return [item.strip() for split_list in [item_.split(';') 
                                            for item_ in attr_name_str.split('::')] 
            for item in split_list]

In [5]:
def split_labels(labels_dataframe):
    split_labels_dict = dict(attribute_id=[], attr_tier1=[], attr_tier2=[], attr_tier3=[])
    for i in range(labels_dataframe.shape[0]):
        tem = split_attr_names(labels['attribute_name'][i])
        split_labels_dict['attribute_id'].append(labels['attribute_id'][i])
        split_labels_dict['attr_tier1'].append(tem[0])
        split_labels_dict['attr_tier2'].append(tem[1])
        try:
            split_labels_dict['attr_tier3'].append(tem[2])
        except:
            split_labels_dict['attr_tier3'].append('None')
    split_labels = pd.DataFrame(split_labels_dict, 
                                columns=['attribute_id', 'attr_tier1', 'attr_tier2', 'attr_tier3'])
    return split_labels

In [6]:
split_labels = split_labels(labels)

In [7]:
split_labels

,attribute_id,attr_tier1,attr_tier2,attr_tier3
0,0,country,afghanistan,None
1,1,country,alamania,None
2,2,country,algeria,None
3,3,country,arabia,None
4,4,country,armenia,None
...,...,...,...,...
3469,3469,tags,writing,None
3470,3470,tags,writing implements,None
3471,3471,tags,zebras,None
3472,3472,tags,zeus,None


In [8]:
def label_indexing(split_labels):
    tier1 = dict()
    tier2 = dict()
    counting_dict = dict()
    attr2indexing = dict()
    indexing2attr = dict()
    label_indexing_list = []
    for i_1, item1 in enumerate(sorted(list(set(list(split_labels['attr_tier1']))))):
        tier1[item1] = i_1
        tier2[item1] = dict()
        list_tem = sorted(list(set(list(split_labels['attr_tier2'][split_labels['attr_tier1']==item1]))))
        for i_2, item2 in enumerate(list_tem):
            tier2[item1][item2] = i_2 + 1
        counting_dict[item1] = np.ones(len(list_tem), dtype='int')
    for idx in range(split_labels.shape[0]):
        tier1_idx = tier1[split_labels['attr_tier1'][idx]]
        tier2_idx = tier2[split_labels['attr_tier1'][idx]][split_labels['attr_tier2'][idx]]
        tier3_idx = counting_dict[split_labels['attr_tier1'][idx]][tier2_idx-1]
        counting_dict[split_labels['attr_tier1'][idx]][tier2_idx-1] += 1
        label_indexing_list.append([tier1_idx, tier2_idx, tier3_idx])
        attr2indexing[split_labels['attribute_id'][idx]] = [tier1_idx, tier2_idx, tier3_idx]
        indexing2attr[str([tier1_idx, tier2_idx, tier3_idx])] = split_labels['attribute_id'][idx]
    labels_indexing_df = split_labels.copy()
    labels_indexing_df['indexing'] = label_indexing_list
    return labels_indexing_df, attr2indexing, indexing2attr
    

In [10]:
labels_indexing_df, attr2indexing, indexing2attr = label_indexing(split_labels)

,attribute_id,attr_tier1,attr_tier2,attr_tier3,indexing
0,0,country,afghanistan,None,"[0, 1, 1]"
1,1,country,alamania,None,"[0, 2, 1]"
2,2,country,algeria,None,"[0, 3, 1]"
3,3,country,arabia,None,"[0, 4, 1]"
4,4,country,armenia,None,"[0, 5, 1]"
...,...,...,...,...,...
3469,3469,tags,writing,None,"[4, 764, 1]"
3470,3470,tags,writing implements,None,"[4, 765, 1]"
3471,3471,tags,zebras,None,"[4, 766, 1]"
3472,3472,tags,zeus,None,"[4, 767, 1]"


In [12]:
data

,id,attribute_ids
0,000040d66f14ced4cdd18cd95d91800f,448 2429 782
1,0000ef13e37ef70412166725ec034a8a,2997 3231 2730 3294 3099 2017 784
2,0001eeb4a06e8daa7c6951bcd124c3c7,2436 1715 23
3,000226398d224de78b191e6db45fd94e,2997 3433 448 782
4,00029c3b0171158d63b1bbf803a7d750,3465 3322 3170 1553 781
...,...,...
142114,fffd00c472c1856f5c8b7e41311e2344,125 783 96 27
142115,fffdb3cde6cfd3b335fc56ecf2f1eae5,2732 3170 448 1559 785
142116,fffdde4325edb076dccea37978c1378a,2813 233 1561 785
142117,fffe25dccc93a5e9e12f26d494dae26f,3095 408 1610 1470 1899 1449


In [14]:
data['attribute_ids'][1].split()

['2997', '3231', '2730', '3294', '3099', '2017', '784']

In [18]:
[attr2indexing[int(attr_id)] for attr_id in data['attribute_ids'][1].split()]

[[4, 292, 1],
 [4, 526, 1],
 [4, 25, 1],
 [4, 589, 1],
 [4, 394, 1],
 [3, 1167, 1],
 [2, 4, 1]]

In [18]:
count_array = np.zeros((data.shape[0], 6), dtype='int')
for i, attr_ids in enumerate(data['attribute_ids']):
    cata_indexing = [labels_indexing_df['indexing'][int(attr_id)][0] 
                     for attr_id in data['attribute_ids'][i].split()]
    for i_ in range(count_array.shape[1]):
        count_array[i, i_] = cata_indexing.count(i_)
for i_tier1, tier1 in enumerate(sorted(list(set(list(split_labels['attr_tier1']))))):
    print('Catagory: %s; minimum count of tagging per item: %s; maximum count of tagging per item: %s' 
          %(tier1, np.amin(count_array, axis=0)[i_tier1], np.amax(count_array, axis=0)[i_tier1]))

Catagory: country; minimum count of tagging per item: 0; maximum count of tagging per item: 3
Catagory: culture; minimum count of tagging per item: 0; maximum count of tagging per item: 4
Catagory: dimension; minimum count of tagging per item: 0; maximum count of tagging per item: 1
Catagory: medium; minimum count of tagging per item: 0; maximum count of tagging per item: 17
Catagory: tags; minimum count of tagging per item: 0; maximum count of tagging per item: 18


In [23]:
data['attribute_ids'][count_array.argmax(axis=0)[0]]

'403 784 13 73 22'

In [24]:
split_labels.iloc[13]

attribute_id         13
attr_tier1      country
attr_tier2        chile
attr_tier3         None
Name: 13, dtype: object

In [25]:
split_labels.iloc[73]

attribute_id         73
attr_tier1      country
attr_tier2         peru
attr_tier3         None
Name: 73, dtype: object

In [26]:
split_labels.iloc[22]

attribute_id         22
attr_tier1      country
attr_tier2      ecuador
attr_tier3         None
Name: 22, dtype: object

In [27]:
import cv2

In [33]:
img  = cv2.imread(path+'train/'+str(data['id'][0])+'.png')

In [34]:
img.shape

(300, 375, 3)

In [14]:
import torch.nn.functional as F
import torch

In [15]:
a = torch.FloatTensor([[1,1,1],[0,1,0]])
b = torch.FloatTensor([[0.5,0.8,0.1],[0.01,0.2,0.5]])

In [20]:
a.long().long()

tensor([[1, 1, 1],
        [0, 1, 0]])

In [57]:
torch.mean(F.binary_cross_entropy(b.view(-1,3), a, reduction='none'), dim=1)

tensor([1.0730, 0.7709])

In [30]:
eg = dict()
eg[1]=0
eg[1]=1

In [31]:
eg

{1: 1}

In [32]:
eg[0] = 0
eg[1] = 1
eg[2] = 2
eg[3] = 3

In [35]:
type(eg.values())

dict_values

In [40]:
aaa = torch.FloatTensor(np.zeros(5))
aaa[2]=1

In [61]:
torch.cat

In [75]:
aaa = torch.FloatTensor([[0.1, 0.6, 0.7, 0.2], [0.3, 0.4, 0.9, 0.23]])
aaa_ = np.zeros(np.array(aaa).shape, dtype='int')
aaa_[:, 2:4][aaa[:, 2:4]>0.5] = 1

In [76]:
aaa_

array([[0, 0, 1, 0],
       [0, 0, 1, 0]])

In [65]:
bbb=[]

In [66]:
bbb += [aaa.view(-1)]

In [70]:
aaa_[:, 0:2]>0.5

array([[False, False],
       [False, False]])

In [78]:
len(([0, 100], [100, 781], [786, 2706], [2706, 3474]))

4

In [48]:
bbb = torch.LongTensor([0])
bbb[0] =2

In [49]:
bbb

tensor([2])

In [50]:
def eg(a):
    return a, a, a, a, a
eg(1)

(1, 1, 1, 1, 1)

In [51]:
eg_tuples = eg(1)

In [52]:
eg_tuples

(1, 1, 1, 1, 1)

In [80]:
from sklearn.metrics import fbeta_score
eg1 = np.array([0,1,1,0,1,0,0])
eg2 = np.array([0,1,0,0,1,0,0])
fbeta_score(eg1, eg2, beta=2)

0.7142857142857142

In [82]:
eg1 = np.array([[0,1,1,0,1,0,0],[0,1,0,0,0,1,0]])
eg2 = np.array([[0,1,0,0,1,0,0],[0,1,0,1,1,1,0]])
[fbeta_score(eg1[i,:], eg2[i,:], beta=2) for i in range(eg1.shape[0])]

NameError: name 'mean' is not defined